<span><h1 style="color:#4987FF">
Cython:
</h1></span>

#### From Cython's documentation;
***"Cython is Python with C data types."***

#### There are a few different ways to use Cython;
#### > From a Jupyter Notebook using Jupyter Magic
#### > Writing a .pyx file to be compiled by Cython

#### Resources for a much better understanding of usage / inner workings of Cython will be at the bottom of this notebook, but in this notebook (and talk) we will only focus on using Cython with Jupyter magic, and we will only learn the basics of it enough to achieve some easy performance boosts.

## Official Website: http://cython.org/

In [1]:
"""
We're just going to jump straight in;
"""

%load_ext cython
import numpy as np
import math
import timeit

In [ ]:
"""
There are a few different ways to declare a Cython-y function:

def func1():
    # Will be visible in pure python and in cython, can implement C/Cython functions

cdef func2():
    # Will *NOT* be visible in pure python, only to other cython functions, can implement C/Cython and/or pure Python
    
cpdef func3():
    # Will be visible from both pure python and cython code, is itself compiled as cython, can imlplement everything

"""

In [27]:
%%cython -a
# -a flag tells it to show us the C that Cython generates for our code, we don't usually care

# This example shows that we can access the C standard library easily wth Cython 

from libc.math cimport sin
# Note: to import C/C++ functions, we use 'cimport' rather than 'import'

# Note in the example below that cython allows us to specify types of variables; this helps it know 
# how to speed things up. We'll make one function directly in C with types specified, and another 
# in python without specifying types, but still calling the C function 'sin' -

cpdef double cython_func(double x):
    return sin(x*x)

def python_func(x):
    return sin(x*x)

In [26]:
%timeit cython_func(5)
%timeit python_func(5)
%timeit np.sin(5*5)

The slowest run took 15.56 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 343 ns per loop
The slowest run took 10.32 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 517 ns per loop
The slowest run took 9.72 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.53 µs per loop


## So we can see that, while not needed, telling Cython the types lets it get an extra bit of a boost - but still either is more efficient than not using the native C function.

## Also note that while that example only shows a smallish (up to ~20x) speedup, it's worth mentioning that we were comparing it to a Numpy function, which is already pretty optimised.

In [31]:
%%cython

"""
Cython doesn't always have a pre-made way of importing C/C++ functions, so you may need to make one.
(note this is not the case in the example below, but the method works anyway so it doesn't matter for the example)
"""

cdef extern from "math.h":
    double cos(double x)
# Here we've just made a cython-only 'cos' function that uses the C cos function
    
cpdef double cython_cos(double x):
    return cos(x*x)
# Here we've made a cython/python type function that uses the 'cos' function we just made
# Note: in case of doubt, check how we imported the sin one in first example; we didn't import the whole math lib

In [32]:
# And just to show it works;
cython_cos(49.4)

-0.7915275406282308

### In the above example, we assume Cython is able to find the 'math' header file, which is part of the stdlib for C++ so no big deal to find. Additionally, we only wanted to grab a function rather than a whole class. 

### Here we look at using a completely custom C++ class (Rectangle.cpp & Rectangle.h, both in Rectangle/ in this directory). 

### Also, while Jupyter usually handles most of the hard parts of Cython silently behind the scenes, it's useful to see what's needed as it needs to be done manually if you ever aren't using jupyter;

---

First we need to make our C/C++ files and such - that part's not Python so I'm leaving all that out. See Rectangle.h/.cpp to see the example I use here if you're interested. The main point is I make a C++ class, Rectangle, with a method, getArea(). 

Next we need to make a rectangle.pyx file - a cython file to act as a bridge between the C++ and Python; it reads from the header (.h) file and outlines what's there, then creates a cython version of the class (I called PyRectangle). 

Code:
```
cdef extern from "Rectangle.h" namespace "shapes":
    cdef cppclass Rectangle:
        Rectangle(int, int, int, int)
        int x0, y0, x1, y1
        int getArea()

cdef class PyRectangle:
    cdef Rectangle *thisptr      
    def __cinit__(self, int x0, int y0, int x1, int y1):
        self.thisptr = new Rectangle(x0, y0, x1, y1)
    def __dealloc__(self):
        del self.thisptr
    def getArea(self):
        return self.thisptr.getArea()
```

Next we need to make a setup.py script to prepare it for use in Python directly (don't actually need to import cython even to use after this script is run). The setup script essentially says "make a new python library thing for us to use, use Cython to compile this .pyx file which needs this .cpp file, and call the new library thing 'rectangle'"

Code:

```
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext

setup(ext_modules = [Extension("rectangle", 
                                  ["rectangle.pyx",
                                   "Rectangle.cpp"],
                                language="c++")],
      cmdclass = {'build_ext': build_ext})
```

Finally, all that's needed is to run the setup script and you have a useable python library - which you can import without involving cython. To run the script in the terminal, enter
```
$ python setup.py build_ext
```

In this example, I call the script in the cell below instead (using some Jupyter magic). Notice that a new 'rectangle.cpp' file is generated automatically when this script runs, with a lot more content - to handle python bindings and such.

The cell below here uses this newly created library thing, note again that there is no mention of C/C++/Cython.

In [8]:
%cd Rectangle/
%run setup.py build_ext -i
%cd ..

# Note in the commands above that we needed to be in the same directory as the files to build them - 
# this is because of how we referenced their locations in the setup.py file.

# Note also that to be able to import from the Rectangle/ subdirectory we had to place a '__init__.py' file
# there in order to tell Python to treat that directory as a module

from Rectangle.rectangle import PyRectangle
r = PyRectangle(1,1,5,5) # params are corners; x0, y0, x1, y1 - so a 4x4 square in this case
print(r.getArea())

/home/luke/my_work/QUB_DW_HighPerformancePython/HighPerformance/Rectangle
running build_ext
skipping 'rectangle.cpp' Cython extension (up-to-date)
/home/luke/my_work/QUB_DW_HighPerformancePython/HighPerformance
16


## So that's actually all we're going to cover for Cython here, other than a bit in Example2
## To use Cython more efficiently, learn some basic C/C++, especially the standard library and data types/structs.

#### Short list of things Cython allows that may be useful to you (not shown here):
* use different compilers
* use different compiler flags
* use using parallelised C/C++ code
* use Cython to port Python code to C/C++
* use Cython to optimise compilation for a particular architecture
* profle Cython functions/methods with Cython's @cython_profile decorator 
